# Orchestration Concepts - Interactive Workshop

## Learning Goals
- Understand the difference between single calls and orchestration
- See how structured outputs become orchestration triggers
- Learn context management strategies across calls

## What is Orchestration?

**The Core Problem**  
Real AI applications need multiple steps:
- "Find me a cheap lunch nearby" requires: location → restaurants → prices → comparison
- Single LLM calls can't access external data or perform calculations
- Orchestration = chaining actions based on LLM decisions

**Single Call vs Orchestration**
```
Single Call:     User Input → LLM → Response (limited)
Orchestration:   User Input → LLM → Tool Call → LLM → Tool Call → Response (powerful)
```

## Setup

First, let's import the required libraries and set up our OpenAI client.

In [ ]:
import json
from openai import OpenAI
from pydantic import BaseModel
from typing import List, Optional

client = OpenAI()

# Concept 1: Structured Outputs as Orchestration Triggers

**The Key Insight**: Structured outputs tell us what to do next

```python
# This structured response contains our next actions:
{
  "items": ["Big Mac", "medium fries", "Coke"],
  "action": "search_prices",  # ← This triggers the next step
  "restaurant": "McDonald's",
  "confidence": 0.95
}
```

In [ ]:
class MealAnalysis(BaseModel):
    """Structured output that contains orchestration instructions"""
    
    items: List[str]
    action: str  # This field determines what happens next!
    restaurant: Optional[str] = None
    confidence: float

In [ ]:
def analyze_meal_order(order: str) -> MealAnalysis:
    """
    CONCEPT: The LLM returns structured data that tells us what to do next
    """
    response = client.responses.parse(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": f"""Analyze this meal order: "{order}"
        
        Break it into individual items and decide the next action.
        Set action to:
        - 'search_database' if it's a common restaurant meal
        - 'search_web' if it's an unusual or specific item
        - 'estimate' if it's too vague to search
        
        Be specific about restaurant chains when mentioned.""",
            }
        ],
        text_format=MealAnalysis,
    )
    return response.output_parsed

### Demo: Structured Outputs as Triggers

Let's see how different meal orders result in different actions being triggered:

In [ ]:
def demo_structured_triggers():
    """Demonstrate how structured outputs trigger different actions"""
    
    print("=== CONCEPT 1: Structured Outputs as Triggers ===\n")
    
    test_orders = [
        "Big Mac meal from McDonald's",
        "homemade chicken sandwich", 
        "some lunch food"
    ]
    
    for order in test_orders:
        print(f"📝 Order: '{order}'")
        analysis = analyze_meal_order(order)
        
        print(f"🔍 Structured Output:")
        print(f"   Items: {analysis.items}")
        print(f"   Action: {analysis.action} ← This determines what happens next!")
        print(f"   Restaurant: {analysis.restaurant}")
        print(f"   Confidence: {analysis.confidence}")
        
        # Show what the action triggers
        if analysis.action == "search_database":
            print("   ✅ Would trigger: Database lookup for restaurant prices")
        elif analysis.action == "search_web":
            print("   ✅ Would trigger: Web search for price information")
        elif analysis.action == "estimate":
            print("   ✅ Would trigger: Price estimation based on similar items")
            
        print("-" * 50)

# Run the demo
demo_structured_triggers()

# Concept 2: Context Management Across Calls

Context must be carefully managed across multiple orchestration steps. Each step receives context from the previous step and potentially modifies it for the next step.

In [ ]:
class PriceContext(BaseModel):
    """Context that accumulates through the orchestration chain"""
    
    original_order: str
    items: List[str]
    prices_found: List[dict] = []
    total_price: Optional[float] = None
    search_method: str = ""

In [ ]:
def mock_database_lookup(item: str) -> Optional[dict]:
    """Mock database that sometimes fails (to demonstrate fallbacks)"""
    
    # Mock database with limited data
    database = {
        "big mac": {"price": 45.0, "currency": "SEK"},
        "medium fries": {"price": 25.0, "currency": "SEK"},
        "french fries": {"price": 25.0, "currency": "SEK"},
        "medium coke": {"price": 20.0, "currency": "SEK"},
        "soft drink": {"price": 20.0, "currency": "SEK"},
    }
    
    return database.get(item.lower())

### Demo: Context Management

Let's trace how context flows through multiple orchestration steps:

In [ ]:
def demo_context_management():
    """Show how context flows and accumulates through orchestration steps"""
    
    print("=== CONCEPT 2: Context Management ===\n")
    
    order = "Big Mac meal"
    
    # Step 1: Initial analysis (creates context)
    print("🔗 Step 1: Parse order and create context")
    analysis = analyze_meal_order(order)
    
    context = PriceContext(
        original_order=order,
        items=analysis.items,
        search_method="starting"
    )
    print(f"   Context created: {context.original_order}")
    print(f"   Items to price: {context.items}")
    
    # Step 2: Try database lookup (context accumulates)
    print("\n🔗 Step 2: Database lookup (context accumulates)")
    for item in context.items:
        price_data = mock_database_lookup(item)
        if price_data:
            context.prices_found.append({"item": item, **price_data})
            print(f"   ✅ Found {item}: {price_data['price']} {price_data['currency']}")
        else:
            print(f"   ❌ No database entry for: {item}")
    
    # Step 3: Calculate total (context completes)
    print("\n🔗 Step 3: Calculate total (context completion)")
    if context.prices_found:
        context.total_price = sum(item["price"] for item in context.prices_found)
        context.search_method = "database"
        print(f"   💰 Total: {context.total_price} SEK")
        print(f"   📊 Method: {context.search_method}")
    
    print(f"\n📦 Final Context State:")
    print(f"   Original: {context.original_order}")
    print(f"   Total Price: {context.total_price}")
    print(f"   Items Priced: {len(context.prices_found)}")
    print("-" * 50)

# Run the demo
demo_context_management()

# Concept 3: Fallback Strategies

Robust orchestration systems may use fallback strategies when primary methods fail. This prevents the system from breaking when external data sources are unavailable.

In [ ]:
def mock_web_search(item: str) -> dict:
    """Mock web search fallback"""
    # In real implementation, this would search the web
    estimated_prices = {
        "chicken sandwich": 65.0,
        "homemade": 35.0,
        "lunch": 50.0
    }
    
    # Find best match or use default
    for key in estimated_prices:
        if key in item.lower():
            return {
                "price": estimated_prices[key],
                "currency": "SEK",
                "source": "web_estimate"
            }
    
    return {
        "price": 50.0,
        "currency": "SEK",
        "source": "default_estimate"
    }

### Demo: Fallback Patterns

Let's see how fallbacks work when our primary data source fails:

In [ ]:
def demo_fallback_patterns():
    """Show how fallbacks work when primary methods fail"""
    
    print("=== CONCEPT 3: Fallback Patterns ===\n")
    
    # Test with item not in our mock database
    item = "chicken sandwich"
    
    print(f"🔍 Pricing: '{item}'")
    
    # Primary: Try database
    print("   1️⃣ Primary: Database lookup")
    db_result = mock_database_lookup(item)
    if db_result:
        print(f"      ✅ Found: {db_result}")
        return db_result
    else:
        print(f"      ❌ Not found in database")
    
    # Fallback: Try web search
    print("   2️⃣ Fallback: Web search")
    web_result = mock_web_search(item)
    print(f"      ✅ Web estimate: {web_result}")
    
    print("\n💡 Key Insight: Fallbacks let us handle missing data gracefully")
    print("   Without fallbacks → System breaks")
    print("   With fallbacks → System provides best available answer")
    print("-" * 50)

# Run the demo
demo_fallback_patterns()

# Key Takeaways

🎓 **What We've Learned:**

1. **Structured outputs contain orchestration instructions** - The LLM doesn't just return data, it returns instructions for what to do next

2. **Context must be carefully managed across calls** - Information flows from step to step and accumulates over time

3. **Fallbacks are essential for robust systems** - Primary methods will fail, so always have backup strategies

4. **Not every task needs orchestration** - Single calls work fine when no external data or multi-step processing is needed

**Next Steps**: In the next session, we'll explore more advanced orchestration patterns including tool calling and agent frameworks.